#### Load Movie reviews Dataset

We will be using data available on Kaggle platform for this exercise. The data is available at https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [ ]:
#Connect Google drive to colab
from google.colab import drive
drive.mount('/gdrive')

Load dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#change file path to point to where you have stored the zip file.
df = pd.read_csv('/gdrive/My Drive/AI-ML/labeledTrainData.tsv.zip', header=0, delimiter="\t", quoting=3)

In [ ]:
print('Number of examples in Dataset: ', df.shape)
df.head()

In [ ]:
df.loc[0, 'review']

Split Data into Training and Test Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

In [ ]:
X_train.shape, X_test.shape

#### Build the Tokenizer

In [ ]:
import tensorflow as tf

In [ ]:
desired_vocab_size = 10000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size, oov_token=32) # num_words -> Vocablury size

In [ ]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [ ]:
#Vocabulary
t.word_index

#### Prepare Training and Test Data

Get the word index for each of the word in the review

In [ ]:
X_train[0:1]

In [ ]:
X_train = t.texts_to_sequences(X_train.tolist())

In [ ]:
print(X_train[0:1])

In [ ]:
X_test = t.texts_to_sequences(X_test)

How many words in each review?

In [ ]:
len(X_train[200])

#### Pad Sequences - Important

In [ ]:
#Define maximum number of words to consider in each review
max_review_length = 300

In [ ]:
#Pad training and test reviews
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_review_length,
                                                        padding='pre')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_review_length, 
                                                       padding='pre')

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train[200]

#### Download Glove model

Check out this [link](https://nlp.stanford.edu/projects/glove/) for available pre-trained Glove models from Standord.

In [ ]:
#This will take few minutes
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#Check if embeddings have been downloaded
!ls -l

In [ ]:
#unzip the file, we get multiple embedding files. We can use either one of them
!unzip glove.6B.zip

We have 4 models in this zip file. '50d' in the filename means that embedding size is 50 (100d means embedding size 100). We can use any one of the 4 models.

In [ ]:
!ls -l

#### Convert Glove to Word2Vec format

In [ ]:
#Install Gensim
!pip install gensim --quiet

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
#Glove file - we are using model with 50 embedding size
glove_input_file = 'glove.6B.50d.txt'

#Name for word2vec file
word2vec_output_file = 'glove.6B.50d.txt.word2vec'

#Convert Glove embeddings to Word2Vec embeddings
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
!ls -l

#### Get Pre-trained Embeddings

Pre-trained Glove model has 400,000 unique words (Vocabulary size). We do not need all the words. Moreover, we have to arrange word embeddings according to word index created by our tokenizers above. So we will extract word embeddings for only the words that we are interested in.

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# Load pretrained Glove model (in word2vec form)
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
#Embedding length based on selected model - we are using 50d here.
embedding_vector_length = 50

Initialize a embedding matrix which we will populate for our vocabulary words.

In [ ]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 50 columns (as embedding size is 50)
embedding_matrix = np.zeros((desired_vocab_size + 1, embedding_vector_length))

Load word vectors for each word in our vocabulary from from Glove pre-trained model

In [ ]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = glove_model[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

We now have word embeddings for our vocabulary words from Glove model. We can now use it in our Model training.

In [ ]:
#embedding_matrix[2]

#### Build Model - Dense Layers

In [ ]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

To handle, pre-trained embeddings, we will use Keras Embedding layer

In [ ]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
          )

Embedding Layer gives us 3D output ->
[Batch_Size , Review Length , Embedding_Size]

In [ ]:
model.output

Add Hidden layers

In [ ]:
#Flatten the data as we will use Dense layers
model.add(tf.keras.layers.Flatten())

#Add Hidden layers (Dense layers)
model.add(tf.keras.layers.Dense(100, activation='relu', input_shape=()))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

Add Output layer

In [ ]:
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

##### Train Model

In [ ]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

#### Building a CNN Model

Start a model

In [ ]:
model2 = tf.keras.Sequential()

Add Embedding layer to handle Word2Vec

In [ ]:
model2.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
          )

Add Conv1D hidden layers : As our text data is 2D (number of words, Embedding size), we will use Conv1D in this case (compared to Conv2D with images which are 3D)

In [ ]:
#Add first convolutional layer
model2.add(tf.keras.layers.Conv1D(32, #Number of filters 
                                 kernel_size=(3), #Size of the filter
                                 strides=1,
                                 activation='relu'))

#normalize data
model2.add(tf.keras.layers.BatchNormalization())

#Add second convolutional layer
model2.add(tf.keras.layers.Conv1D(64, kernel_size=(3), strides=2))
model2.add(tf.keras.layers.ReLU())

#normalize data
model2.add(tf.keras.layers.BatchNormalization())

In [ ]:
#Use Global Average Pooling
model2.add(tf.keras.layers.GlobalAveragePooling1D())

#Output layer
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
#Compile the model
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
model2.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))